In [4]:
import pandas as pd
new_news1 = pd.read_csv('./csv/2017전처리멀쩡.csv',encoding='cp949')
# new_news2 = pd.read_excel('./new_news/NewsResult_20200701-20201231.xlsx')
# new_news = pd.concat([new_news1,new_news2], axis = 0)

In [5]:
data_mt = new_news1.copy()
data_mt=data_mt.reset_index(drop=True)

In [27]:
# import pandas as pd
# data_mt = pd.read_csv('./ESG기사2.csv')
# data_mt2 = pd.read_csv('./data_mt2.csv')
# data_mt.drop('Unnamed: 0' , axis = 1 , inplace= True)
# data_mt2.drop('Unnamed: 0' , axis = 1 , inplace= True)

In [6]:
data_mt['경영진에게'] = data_mt['경영진에게'].apply(str)
target_texts = data_mt['경영진에게'].tolist()
target_texts[0]

'경영진이 문제이고 바뀌지 않는 한 변화가 될 수 없는 기업이다'

In [7]:
import sys
from time import sleep
# import backoff
from googletrans import Translator

class MAXTranslator:
    def __init__(self):
        self.client = Translator()
        self.sleep_in_between_translations_seconds = 0.1
        self.source_language = "ko"
        self.max_chunk_size = 5000

    def __createChunks(self, corpus):
        chunks = [corpus[i:i + self.max_chunk_size] for i in range(0, len(corpus), self.max_chunk_size)]
        return chunks

    def __sleepBetweenQuery(self):
        # print('Sleeping for {}s after translation query..'.format(self.sleep_in_between_translations_seconds))
        sleep(self.sleep_in_between_translations_seconds)

    # @backoff.on_exception(backoff.expo, Exception, max_tries=2)
    def Translate(self, content, dest_language_code):
        try:
            #print(target_texts.index(content))
            # print('Attempting to translate to lang={}'.format(dest_language_code))
            if len(content) > self.max_chunk_size:
                # print('Warning: Content is longer than allowed size of {}, breaking into chunks'.format(self.max_chunk_size))
                results_list = []
                concatenated_result = ""

                original_chunks = self.__createChunks(content)
                for i in original_chunks:
                    r = self.client.translate(i, dest=dest_language_code, src=self.source_language)
                    self.__sleepBetweenQuery()
                    results_list.append(r.text)

                for i in results_list:
                    concatenated_result += i

                return concatenated_result
            else:
                res = self.client.translate(content, dest=dest_language_code, src=self.source_language)
                self.__sleepBetweenQuery()
                return res.text
        except Exception as e:
            print(e)
            raise e

In [8]:
TEXT_MAX_LENGTH = 5000  # google translate allows maximum size of 5000 for one request

def make_chunks(sentence_lst):
    """
    Chunk is a sentence that is not longer than TEXT_MAX_LENGTH
    By looping the list of sentences, we will make a new chunk which is not longer than TEXT_MAX_LENGTH, while as long as possible
    """
    input_chunk_lst = []
    chunk = ""
    for sentence in sentence_lst:
        sentence = sentence.strip()
        sentence = sentence + "\r\n"
        if len((chunk.rstrip() + sentence).encode('utf-8')) > TEXT_MAX_LENGTH:
            input_chunk_lst.append(chunk.rstrip())
            chunk = sentence
        else:
            chunk = chunk + sentence
    input_chunk_lst.append(chunk.rstrip())
    return input_chunk_lst

In [9]:
import time
import argparse
from tqdm import tqdm

def google_translate(sentence_lst):
        input_chunk_lst = make_chunks(sentence_lst)
        trans = MAXTranslator()
        translated_sentence_lst = []

        for kr_chunk in tqdm(input_chunk_lst):
            en_chunk = trans.Translate(content=kr_chunk,dest_language_code='en')
            en_sentences = en_chunk.split("\r\n")
            if en_sentences[-1] == "":
                en_sentences = en_sentences[:-1]
            translated_sentence_lst.extend(en_sentences)
        return translated_sentence_lst

In [10]:
translate_sentence_lst = google_translate(target_texts) # 5000자 넘는 corpus 오류

100%|██████████| 541/541 [13:02<00:00,  1.45s/it]


In [11]:
result_news3 = pd.DataFrame(translate_sentence_lst,columns=['특성추출_번역'])
data_mt = pd.concat([data_mt,result_news3],axis=1)
data_mt

,총점,회사,날짜,별점,요약,장점,단점,경영진에게,특성추출_번역
0,3.0,(주)메타바이오메드,2022. 09,1점,"""오래된 지방의 중소 제조기업 그 자체 정치질 잘하고 일 편하게 다니고 싶은 사람에...",연차 사용 자유\n점심 제공\n그 외에는 딱히 장잠이 없다,회사 내 분위기가 어수선함\n고인물파티\n실무자 없음\n체계없음,경영진이 문제이고 바뀌지 않는 한 변화가 될 수 없는 기업이다,It is a company that does not change unless ma...
1,3.0,(주)메타바이오메드,2022. 09,3점,"""워라밸은 최고이나 과연 계속 다녀도 괜찮을까?라는 느낌이드는 회사""","왠만하면 정시 칼퇴근, 연차사용 매우 자유로움, 업무 별로 안힘듬, 직원들도 대부분...","낮은 임금(일한 만큼 받는거 같긴함?)으로 퇴사자가 잦음, 회사의 전망이 별로 좋지...","직원들에게 돈을 너무 아낀다, 성과를 냈으면 그만한 보상이 있어야하는데 너무 인색하...",If you have done so much money to your employe...
2,3.0,(주)메타바이오메드,2022. 08,2점,"""연봉 생각안하고 워라벨을 중시하면 다니기 좋음 복지는 그저 그런편 진급해도 연봉 ...",연차를 윗사람 눈치안보고 쓸수있고 종치면 바로 퇴근이라 워라벨 좋음,연봉이 너무 적음 바이오 업계에서는 거의 최하위 수준으로 생각,공장 증축보다는 사람을 먼저 생각하며 연봉 좀 더 늘려줬으면 좋겠음,I hope to increase the salary more than the fa...
3,3.0,(주)메타바이오메드,2022. 08,4점,"""워라벨을 중요시 여긴다면 정말 좋은회사이다. 모두 칼퇴함""",워라벨이좋고 직원분들이 다들 좋으시다. 회사분위기가 나쁜편은아님,연봉이 높지않고 출근시간이 매우 빠르다. 기숙사에서 회사가멀다.,직원들의 불만을 듣지만말고 해결해줬음 좋겠다. 권위적인 체계가 없어졌음 좋겠다.,I hope you solved it without listening to the ...
4,3.0,(주)메타바이오메드,2022. 08,3점,"""생각없이 무난하게 다니기 좋은곳이면 여기가 최고인 기업""",아침 점심 제공 워라벨 야근강요 없음 칼퇴보장 주말휴무,적은급여 돈에 대한 부담있으면 이게 단점 업무안하고 농땡이 치는 몇몇 올드들,시급도 올라가고 사람구하기 힘든시기니 급여쪽 상승을 더더욱 진지하게 생각하시길,It is a time to go up the hourly wage and it i...
...,...,...,...,...,...,...,...,...,...
19206,2.5,제이씨케미칼(주),2022. 04,2점,"""뭔가 애매한 사이즈 애매한 연봉 애매한 전망 모두가 다 애매하다""",임원들이 그래도 구시대적이진 않음\n야근거의 전무\n야근수당 있음\n구내식당 맛있음...,시스템부재\n전자결재 없음\n종이와 엑셀이 날아다님\n전반적으로 올드한 시스템\n명...,시스템 변화 안하면 여기는 나중에 경쟁사한테 뒤떨어집니다.\n전체적인 직원들 연봉상...,"If you don't change the system, this will be i..."
19207,2.5,제이씨케미칼(주),2022. 02,2점,"""바이오디젤 및 바이오중유 제조업체로 매우 경직된 분위기 속에서 근무""",바이오연료 제품 및 시장에 대해서 지식을 쌓을 수 있음,연봉이 낮으며 매우 수직적인 분위기 속에서 근무해야 함,직원들이 적극적으로 일할 수 있는 분위기를 마련해야 함,It is necessary to establish an atmosphere whe...
19208,2.5,제이씨케미칼(주),2020. 02,4점,"""업무강도에 비해 보수 및 복지가 좋은 회사""",기본적으로 성과급이 어느정도 보장되어 있고 사업이 안정화 되어있어 보수적인 측면에서...,SK출신과 비출신간의 차등이 있어 비출신의 경우 괴리감이 있을수 있다.,SK출신이 아니더라도 더 뛰어난 업무성과를 기대할수 있으니 출신이 아닌 능력으로 평...,"Even if you are not from SK, you can expect be..."
19209,2.5,제이씨케미칼(주),2020. 02,3점,"""국내 바이오에너지 회사 중 유일하게 수직계열화를 이룬 회사\n해외 사업장 보유하여,""","국내 유일 바이오에너지 수직계열사, 추가적인 투자가 지속되고 있음.\n모회사가 석유...","제조업 베이스이다보니 보수적 분위기가 상당부분 있음.\n전반적으로 분위기 침체, 가...","하나됨을 외치지만, 정작 본인들께서 분위기 경직되게 만드는 상황이 많음.\n아울러,...","Although they shout one, they often make them ..."


In [16]:
data_mt.to_csv('./csv/2017전처리멀쩡_영문번역.csv')

In [17]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

df_final_11 = pd.read_csv('./csv/2017전처리멀쩡_영문번역.csv')
df_final_11['특성추출_번역'] = df_final_11['특성추출_번역'].astype('str')
# tone nlp
result_t_label11 = []
result_t_score11 = [] 
for i in range(len(df_final_11['특성추출_번역'])) :
    print(i+1,"/", len(df_final_11['특성추출_번역']))
    result11 = nlp(df_final_11['특성추출_번역'][i])
    result_t_label11.append(result11[0]['label'])
    result_t_score11.append(result11[0]['score'])
result_t_df_label11 = pd.DataFrame(result_t_label11,columns=['label'])
result_t_df_score11 = pd.DataFrame(result_t_score11,columns=['score'])

df_final_tone11 = pd.concat([df_final_11,result_t_df_label11,result_t_df_score11],axis=1)

1 / 19211
2 / 19211
3 / 19211
4 / 19211
5 / 19211
6 / 19211
7 / 19211
8 / 19211
9 / 19211
10 / 19211
11 / 19211
12 / 19211
13 / 19211
14 / 19211
15 / 19211
16 / 19211
17 / 19211
18 / 19211
19 / 19211
20 / 19211
21 / 19211
22 / 19211
23 / 19211
24 / 19211
25 / 19211
26 / 19211
27 / 19211
28 / 19211
29 / 19211
30 / 19211
31 / 19211
32 / 19211
33 / 19211
34 / 19211
35 / 19211
36 / 19211
37 / 19211
38 / 19211
39 / 19211
40 / 19211
41 / 19211
42 / 19211
43 / 19211
44 / 19211
45 / 19211
46 / 19211
47 / 19211
48 / 19211
49 / 19211
50 / 19211
51 / 19211
52 / 19211
53 / 19211
54 / 19211
55 / 19211
56 / 19211
57 / 19211
58 / 19211
59 / 19211
60 / 19211
61 / 19211
62 / 19211
63 / 19211
64 / 19211
65 / 19211
66 / 19211
67 / 19211
68 / 19211
69 / 19211
70 / 19211
71 / 19211
72 / 19211
73 / 19211
74 / 19211
75 / 19211
76 / 19211
77 / 19211
78 / 19211
79 / 19211
80 / 19211
81 / 19211
82 / 19211
83 / 19211
84 / 19211
85 / 19211
86 / 19211
87 / 19211
88 / 19211
89 / 19211
90 / 19211
91 / 19211
92 / 192

In [19]:
df_final_tone11.to_csv('./정상2017잡플레닛tone.csv')